# Neural Network Post-Outlier Removal

This notebook contains the updated model results after removing outliers in the dataset.

---

In [ ]:
# Read in csv files
import io
from google.colab import files

# Foundational packages
import pandas as pd
import numpy as np

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Neural Network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, GRU, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Scoring
from sklearn.metrics import r2_score

# Upload Files

In [ ]:
# Upload clean no outlier X train csv file
uploaded1 = files.upload()

Saving outlier_clean_X_train.csv to outlier_clean_X_train.csv


In [ ]:
# Upload clean no outlier y train csv file
uploaded2 = files.upload()

Saving outlier_clean_y_train.csv to outlier_clean_y_train.csv


In [ ]:
# Upload clean no outlier X test csv file
uploaded3 = files.upload()

Saving outlier_clean_X_test.csv to outlier_clean_X_test.csv


In [ ]:
# Upload clean no outlier y test csv file
uploaded4 = files.upload()

Saving outlier_clean_y_test.csv to outlier_clean_y_test.csv


In [ ]:
# Read in all of the csv files
X_train = pd.read_csv(io.BytesIO(uploaded1['outlier_clean_X_train.csv']), index_col=0)
y_train = pd.read_csv(io.BytesIO(uploaded2['outlier_clean_y_train.csv']), index_col=0)
X_test = pd.read_csv(io.BytesIO(uploaded3['outlier_clean_X_test.csv']), index_col=0)
y_test = pd.read_csv(io.BytesIO(uploaded4['outlier_clean_y_test.csv']), index_col=0)

In [ ]:
# Take a look at my dataframes
display(X_train)
display(y_train.head())
display(X_test.head())
display(y_test.head())

,latitude,longitude,accommodates,bathrooms,bedrooms,beds,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,"('location_Central Oahu',)","('location_East Honolulu',)","('location_Ewa',)","('location_Hamakua',)","('location_Hana',)","('location_Hanapepe-Eleele',)","('location_Kapaa-Wailua',)","('location_Kau',)","('location_Kihei-Makena',)","('location_Koloa-Poipu',)","('location_Koolauloa',)","('location_Koolaupoko',)","('location_Lahaina',)","('location_Lanai',)","('location_Lihue',)","('location_Makawao-Pukalani-Kula',)","('location_Molokai',)","('location_North Hilo',)","('location_North Kohala',)","('location_North Kona',)","('location_North Shore Kauai',)","('location_North Shore Oahu',)","('location_Paia-Haiku',)","('location_Primary Urban Center',)","('location_Puna',)","('location_South Hilo',)","('location_South Kohala',)",...,"('amenity_Ricemaker',)","('amenity_Roku',)","('amenity_Room-darkeningshades',)","('amenity_Safe',)","('amenity_Selfcheck-in',)","('amenity_Shampoo',)","('amenity_Sharedgardenorbackyard',)","('amenity_Sharedgyminbuilding',)","('amenity_Sharedhottub',)","('amenity_Sharedoutdoorheatedpool',)","('amenity_Sharedoutdoorpool',)","('amenity_Showergel',)","('amenity_Singlelevelhome',)","('amenity_Smartlock',)","('amenity_Smokealarm',)","('amenity_Soundsystem',)","('amenity_Stainlesssteelelectricstove',)","('amenity_Stainlesssteeloven',)","('amenity_StartUpSupplyOnlyconditioner',)","('amenity_StartUpSupplyOnlyshampoo',)","('amenity_Stove',)","('amenity_TVwithNetflix',)","('amenity_TVwithstandardcable',)","('amenity_Tablecornerguards',)","('amenity_Toaster',)","('amenity_Trashcompactor',)","('amenity_Washer',)","('amenity_Washer\\u2013\\u00a0Inbuilding',)","('amenity_Washer\\u2013\\u00a0Inunit',)","('amenity_Waterfront',)","('amenity_Wifi\\u2013100Mbps',)","('amenity_Wifi\\u2013200Mbps',)","('amenity_Wifi\\u2013300Mbps',)","('amenity_Wifi\\u2013400Mbps',)","('amenity_WindowACunit',)","('amenity_Windowguards',)","('amenity_Wineglasses',)","('amenity_premiumcable',)","('amenity_standardcable',)",sentiment_polarity
15336,21.29081,-157.83992,3,1.0,1,1,89,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.179459
4116,19.62870,-155.98652,6,2.0,3,4,100,10,10,10,10,10,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.500000
14298,20.97027,-156.67791,6,2.0,2,2,100,10,10,10,10,10,8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.466667
9523,19.55464,-155.96514,8,3.0,3,3,100,9,10,10,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.313273
3836,20.92296,-156.69455,6,2.0,2,3,100,9,10,10,10,10,10,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.298500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10955,22.03625,-159.33769,6,2.0,2,5,93,10,9,10,10,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.351646
905,22.21884,-159.49600,4,1.0,1,2,80,9,8,9,8,10,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.283618
5192,21.87091,-159.44859,4,2.0,2,3,100,10,10,8,10,10,10,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0

,price
15336,99.0
4116,130.0
14298,261.0
9523,320.0
3836,519.0


,latitude,longitude,accommodates,bathrooms,bedrooms,beds,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,"('location_Central Oahu',)","('location_East Honolulu',)","('location_Ewa',)","('location_Hamakua',)","('location_Hana',)","('location_Hanapepe-Eleele',)","('location_Kapaa-Wailua',)","('location_Kau',)","('location_Kihei-Makena',)","('location_Koloa-Poipu',)","('location_Koolauloa',)","('location_Koolaupoko',)","('location_Lahaina',)","('location_Lanai',)","('location_Lihue',)","('location_Makawao-Pukalani-Kula',)","('location_Molokai',)","('location_North Hilo',)","('location_North Kohala',)","('location_North Kona',)","('location_North Shore Kauai',)","('location_North Shore Oahu',)","('location_Paia-Haiku',)","('location_Primary Urban Center',)","('location_Puna',)","('location_South Hilo',)","('location_South Kohala',)",...,"('amenity_Ricemaker',)","('amenity_Roku',)","('amenity_Room-darkeningshades',)","('amenity_Safe',)","('amenity_Selfcheck-in',)","('amenity_Shampoo',)","('amenity_Sharedgardenorbackyard',)","('amenity_Sharedgyminbuilding',)","('amenity_Sharedhottub',)","('amenity_Sharedoutdoorheatedpool',)","('amenity_Sharedoutdoorpool',)","('amenity_Showergel',)","('amenity_Singlelevelhome',)","('amenity_Smartlock',)","('amenity_Smokealarm',)","('amenity_Soundsystem',)","('amenity_Stainlesssteelelectricstove',)","('amenity_Stainlesssteeloven',)","('amenity_StartUpSupplyOnlyconditioner',)","('amenity_StartUpSupplyOnlyshampoo',)","('amenity_Stove',)","('amenity_TVwithNetflix',)","('amenity_TVwithstandardcable',)","('amenity_Tablecornerguards',)","('amenity_Toaster',)","('amenity_Trashcompactor',)","('amenity_Washer',)","('amenity_Washer\\u2013\\u00a0Inbuilding',)","('amenity_Washer\\u2013\\u00a0Inunit',)","('amenity_Waterfront',)","('amenity_Wifi\\u2013100Mbps',)","('amenity_Wifi\\u2013200Mbps',)","('amenity_Wifi\\u2013300Mbps',)","('amenity_Wifi\\u2013400Mbps',)","('amenity_WindowACunit',)","('amenity_Windowguards',)","('amenity_Wineglasses',)","('amenity_premiumcable',)","('amenity_standardcable',)",sentiment_polarity
5341,20.95209,-156.68902,7,3.0,3,5,100,10,10,10,10,10,10,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0.366563
12634,20.76369,-155.99276,2,1.0,1,2,94,10,10,10,10,10,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.360092
8947,21.28854,-157.80368,4,1.0,2,2,99,10,10,10,10,10,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.370724
10258,22.22768,-159.47510,4,2.0,2,3,88,10,9,10,10,10,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.380278
9513,20.92746,-156.69267,2,1.0,1,1,100,10,10,9,10,10,9,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.461886


,price
5341,492.0
12634,57.0
8947,150.0
10258,210.0
9513,173.0


## Exploring Neural Networks w/o Dimension Reduction

Let's take a look at a simple neural network model and see if there are any model adjustments that need to be done. As with many models, neural networks benefit from scaling of the data. I will use the `MinMaxScaler` to keep the values between 0 and 1 to maintain the distance between the variables the same, although which scaler I use likely would not greatly affect my outcomes as my features are mostly encoded.

In [ ]:
# Scale the data
minmaxscaler = MinMaxScaler().fit(X_train)
X_train = minmaxscaler.transform(X_train)
X_test = minmaxscaler.transform(X_test)

I will use 2 hidden layers with 50 nodes each and my output layer to generate predictions. In terms of the activation, `relu` will be used to get a quick output.

In [ ]:
# Create a new sequential model
model = keras.Sequential()

# Declare the hidden layers
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(64, activation="relu"))

# Declare the output layer
model.add(layers.Dense(1))

# Compile the model
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError()
)

In [ ]:
# Run the model
history = model.fit(X_train, y_train, epochs=10, verbose=2, validation_split=0.2)

Epoch 1/10
301/301 - 1s - loss: 102.1030 - val_loss: 68.5841
Epoch 2/10
301/301 - 0s - loss: 63.5643 - val_loss: 60.8000
Epoch 3/10
301/301 - 0s - loss: 58.8153 - val_loss: 57.9631
Epoch 4/10
301/301 - 0s - loss: 56.3479 - val_loss: 56.1393
Epoch 5/10
301/301 - 0s - loss: 54.7093 - val_loss: 55.0258
Epoch 6/10
301/301 - 0s - loss: 53.5339 - val_loss: 54.2697
Epoch 7/10
301/301 - 0s - loss: 52.7224 - val_loss: 54.2230
Epoch 8/10
301/301 - 0s - loss: 52.1031 - val_loss: 53.5360
Epoch 9/10
301/301 - 0s - loss: 51.6657 - val_loss: 53.2281
Epoch 10/10
301/301 - 0s - loss: 51.1451 - val_loss: 53.1896


In [ ]:
# Take a look at the loss and get prediction score
training_loss = history.history["loss"][-1]
test_results = model.evaluate(X_test, y_test, verbose=0)

print(f"Train Loss: {training_loss:.4f}")
print(f"Test Loss: {test_results:.4f}")

Train Loss: 51.1451
Test Loss: 51.7894


In [ ]:
# R2 Score
y_predict = model.predict(X_test)
print(f"R2 Score: {r2_score(y_test, y_predict)}")

R2 Score: 0.569872618546112


This time around, the $R^2$ score is relatively good compared to the previous notebook which included outliers. Let's see how the other models will perform.

In [ ]:
# Create a new sequential model
model2 = keras.Sequential()

# Declare the hidden layers
model2.add(layers.Dense(512, activation="relu"))
model2.add(Dropout(0.10))
model2.add(layers.Dense(512, activation="relu"))
model2.add(Dropout(0.10))
model2.add(layers.Dense(512, activation="relu"))
model2.add(Dropout(0.10))
model2.add(layers.Dense(512, activation="relu"))
model2.add(Dropout(0.10))

# Declare the output layer
model2.add(layers.Dense(1))

# Compile the model
model2.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError()
)

In [ ]:
# Run the model
history2 = model2.fit(X_train, y_train, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
301/301 [==============================] - 4s 13ms/step - loss: 86.4996 - val_loss: 56.8109
Epoch 2/10
301/301 [==============================] - 4s 13ms/step - loss: 55.5820 - val_loss: 54.4131
Epoch 3/10
301/301 [==============================] - 4s 13ms/step - loss: 51.9120 - val_loss: 51.0352
Epoch 4/10
301/301 [==============================] - 4s 12ms/step - loss: 51.0474 - val_loss: 52.3347
Epoch 5/10
301/301 [==============================] - 4s 12ms/step - loss: 48.1088 - val_loss: 50.0400
Epoch 6/10
301/301 [==============================] - 4s 12ms/step - loss: 47.6430 - val_loss: 51.9601
Epoch 7/10
301/301 [==============================] - 4s 12ms/step - loss: 46.1515 - val_loss: 53.2827
Epoch 8/10
301/301 [==============================] - 4s 12ms/step - loss: 45.0696 - val_loss: 50.9861
Epoch 9/10
301/301 [==============================] - 4s 12ms/step - loss: 44.0098 - val_loss: 48.9466
Epoch 10/10
301/301 [==============================] - 4s 13ms/step - los

In [ ]:
# Take a look at the loss and get prediction score
training_loss = history2.history["loss"][-1]
test_results = model2.evaluate(X_test, y_test, verbose=0)

print(f"Train Loss: {training_loss:.4f}")
print(f"Test Loss: {test_results}")

Train Loss: 44.0891
Test Loss: 48.2728157043457


In [ ]:
# R2 Score
y_predict = model2.predict(X_test)
print(f"R2 Score: {r2_score(y_test, y_predict)}")

R2 Score: 0.6077599345806604


Like the previous notebook, the $R^2$ score has increased once again but the model seems to slowly overfit to the training data as the number of epochs increase past 4.

In [ ]:
# Create a new sequential model
model3 = keras.Sequential()

# Declare the hidden layers
model3.add(layers.Dense(512, activation="relu"))
model3.add(Dropout(0.10))
model3.add(layers.Dense(256, activation="relu"))
model3.add(Dropout(0.10))
model3.add(layers.Dense(512, activation="relu"))
model3.add(Dropout(0.10))
model3.add(layers.Dense(256, activation="relu"))
model3.add(Dropout(0.10))
model3.add(layers.Dense(128, activation="relu"))
model3.add(Dropout(0.10))

# Declare the output layer
model3.add(layers.Dense(1))

# Compile the model
model3.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError()
)

In [ ]:
# Run the model
history3 = model3.fit(X_train, y_train, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10
301/301 [==============================] - 3s 9ms/step - loss: 88.6032 - val_loss: 55.6205
Epoch 2/10
301/301 [==============================] - 3s 9ms/step - loss: 57.1598 - val_loss: 53.1406
Epoch 3/10
301/301 [==============================] - 3s 9ms/step - loss: 55.1414 - val_loss: 52.1562
Epoch 4/10
301/301 [==============================] - 3s 9ms/step - loss: 50.8755 - val_loss: 51.0473
Epoch 5/10
301/301 [==============================] - 3s 9ms/step - loss: 51.4749 - val_loss: 50.1995
Epoch 6/10
301/301 [==============================] - 3s 9ms/step - loss: 50.5949 - val_loss: 49.8181
Epoch 7/10
301/301 [==============================] - 3s 9ms/step - loss: 47.9503 - val_loss: 51.5147
Epoch 8/10
301/301 [==============================] - 3s 9ms/step - loss: 47.1492 - val_loss: 49.5535
Epoch 9/10
301/301 [==============================] - 3s 9ms/step - loss: 46.5223 - val_loss: 52.3293
Epoch 10/10
301/301 [==============================] - 3s 9ms/step - loss: 46.1294

In [ ]:
# Take a look at the loss and get prediction score
training_loss = history3.history["loss"][-1]
test_results = model3.evaluate(X_test, y_test, verbose=0)

print(f"Train Loss: {training_loss:.4f}")
print(f"Test Loss: {test_results}")

Train Loss: 46.0605
Test Loss: 47.73527908325195


In [ ]:
# R2 Score
y_predict_test = model3.predict(X_test)
print(f"R2 Score: {r2_score(y_test, y_predict_test)}")

y_predict_train = model3.predict(X_train)
print(f"R2 Score: {r2_score(y_train, y_predict_train)}")

R2 Score: 0.62919361494717
R2 Score: 0.6879225337671977


This model has performed the best thus far and seems to begin to plateau, but to be safe I will increase the number of epochs and set an early stop if this is the final model.

## Exploring Neural Networks w/ PCA

In [ ]:
# Use PCA to reduce the data
pca = PCA(n_components=5)
pca.fit(X_train)
transformed_X_train = pca.transform(X_train)
transformed_X_test = pca.transform(X_test)

In [ ]:
# Rerun neural network on transformed data
# Create a new sequential model
PCA_transformed_model = keras.Sequential()

# Declare the hidden layers
PCA_transformed_model.add(layers.Dense(64, activation="relu"))
PCA_transformed_model.add(layers.Dense(32, activation="relu"))
PCA_transformed_model.add(layers.Dense(64, activation="relu"))
PCA_transformed_model.add(layers.Dense(32, activation="relu"))
PCA_transformed_model.add(layers.Dense(64, activation="relu"))

# Declare the output layer
PCA_transformed_model.add(layers.Dense(1))

# Compile the model
PCA_transformed_model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss=keras.losses.MeanAbsoluteError()
)

In [ ]:
PCA_history = PCA_transformed_model.fit(transformed_X_train, y_train, epochs=10, verbose=2, validation_split=0.2)

Epoch 1/10
301/301 - 1s - loss: 100.1020 - val_loss: 75.9563
Epoch 2/10
301/301 - 0s - loss: 73.9860 - val_loss: 73.7498
Epoch 3/10
301/301 - 0s - loss: 72.6015 - val_loss: 73.1014
Epoch 4/10
301/301 - 0s - loss: 71.7856 - val_loss: 72.2319
Epoch 5/10
301/301 - 0s - loss: 71.1786 - val_loss: 71.8418
Epoch 6/10
301/301 - 0s - loss: 70.5858 - val_loss: 71.6013
Epoch 7/10
301/301 - 0s - loss: 70.3464 - val_loss: 71.3921
Epoch 8/10
301/301 - 0s - loss: 70.1880 - val_loss: 70.4620
Epoch 9/10
301/301 - 0s - loss: 69.9455 - val_loss: 70.4304
Epoch 10/10
301/301 - 0s - loss: 69.6057 - val_loss: 71.0056


In [ ]:
# Take a look at the loss and get prediction score
training_loss = PCA_history.history["loss"][-1]
test_results = PCA_transformed_model.evaluate(transformed_X_test, y_test, verbose=0)

print(f"Train Loss: {training_loss:.4f}")
print(f"Test Loss: {test_results:.4f}")

Train Loss: 69.6057
Test Loss: 68.6395


In [ ]:
# R2 Score
y_predict = PCA_transformed_model.predict(transformed_X_test)
print(f"R2 Score: {r2_score(y_test, y_predict)}")

R2 Score: 0.29117305165109053


Once again, using PCA on this model has proven to be unneccessary for this particular problem and dataset when using a neural network.

## Final Model

Because the neural network was found to be the best model out of all the models that were used, I will use the neural network model with 5 hidden layers as my final model in the main notebook.